<a href="https://colab.research.google.com/github/YeoniH/geometry-driven-hyperuniformity/blob/main/phase_angle_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this Notebook, we will see how the phase angle correlations develop along the evolution of a point pattern via Lloyd iterations.

The *hexatic order parameter* $\psi_6$ of a point $\mathbf{x}$ can be defined as follows:
\begin{equation}
  \psi_6(\mathbf{x}) = \frac{1}{n_\mathbf{x}} \sum_{\mathbf{y}}e^{6i\theta_{xy}} = |\psi_6(\mathbf{x})|e^{i\Theta(\mathbf{x})},
\end{equation}
which is a complex number with magnitude $|\psi_6(\mathbf{x})|\leq 1$ and phase angle $\Theta(\mathbf{x}) \in [-\pi,\pi]$, which indicates the average bond orientation.
The hexatic order of a point configuration can be assessed by the average hexatic order of the points contained.

To proceed with the followings, please copy the Notebook into your local/Google Drive and run the cells below.

First, let us mount a drive called `gdrive`.

(You may need to allow Google Drive for desktop additional access to your Google account.)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Then we create a folder named `my_projects` and change directory (via shell command `cd`) to the newly created folder.

In [ ]:
!mkdir my_projects
%cd my_projects

We will need to clone the following GitHub repository, where all the codes we need are stored.

In [ ]:
!git clone https://github.com/YeoniH/geometry-driven-hyperuniformity.git

Whenever you run this Notebook afresh, please make sure you run the following line to pull all the new updates in the GitHub repository.

In [ ]:
!git pull

Now, we will install one of the fundamental libraries, called `freud-analysis`, for our computational analyses of an evolving point configuration.

In [ ]:
!pip install freud-analysis

Just to double check if we have all we need, we use `ls` (list command).

(If you see `geometry-driven-hyperuniformity` and `/content/my_projects/geometry-driven-hyperuniformity` as output, then you're good to go.)

In [ ]:
!ls
%cd geometry-driven-hyperuniformity

In the following cell, we define a new function to generate a realisation of Poisson point process with `density` value and side length `L` of the
(square) simulation box as input.

In [ ]:
from os import system
import numpy as np

def generate_2d_poisson_pp(density, L):
  """
  Args:
    density: rate, i.e., number of points per unit volume
    L: side length of the square simulation box
  """
  mean = density * (L**2)
  N = np.random.poisson(mean)

  X = np.random.uniform(-L/2, L/2, N)
  Y = np.random.uniform(-L/2, L/2, N)

  coord = np.zeros((N, 3))
  for i in range(N):
    coord[i] = [X[i], Y[i], 0.0]
  return coord

Now, let us generate a new point pattern using the function `generate_2d_poisson_pp()` that we just defined.

(You can play around with different values for `L`, but it is recommended to use `L` larger than 50 to obtain a nice result at the end, which is otherwise make it hard to see the develpment of hexagonal domains and the correesponding correlation length.)

In [ ]:
L = 20
init_points = generate_2d_poisson_pp(density=1, L=L)
num_points = len(init_points)

Let us print out how many numbers the newly generated system (point pattern/configuration) has and also see how it looks like.

In [ ]:
print("There are", num_points, "points")

import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.tick_params(axis='both', labelsize=11)
ax.set_aspect('equal', 'box')
ax.scatter(init_points[:, 0], init_points[:, 1], s=2, color="k")
plt.show()

Given the initial point pattern, we run the Lloyd's algorithm to drive the system into a frozen converged state (effectively hyperuniform state) and plot the temporal evolution of the total quantizer energy of the system along the Lloyd dynamic.

You can choose the variable `num_iter` as small or large as you want, but it is highly recommendable to run it sufficiently (say $\geq$ 10000 iterations for systems with more than 5000 points) to attain hyperuniformity.

In [ ]:
from Analysis import PeriodicVoro as PV
from tqdm import tqdm

dim = 2
num_iter = 5000   # number of iterations steps of the Lloyd's algorithm

init_config = PV(dim=dim, points=init_points, L=L, step=0)
total_qe = [init_config.compute_total_quantizer_energy()]

points = init_points
for t in tqdm(range(num_iter+1)):
  config = PV(dim=dim, points=points, L=L, step=t)
  points = config.update_by_lloyd_algorithm()
  total_qe.append(points.compute_total_quantizer_energy())
final_points = points

plt.plot(list(range(num_iter)), total_qe)
plt.xlabel('Lloyd step', fontsize=12)
plt.ylabel('Total quantizer energy', fontsize=12)
plt.show()

We can now plot the Voronoi landscape of the final point pattern obtained above (at Lloyd step `num_iter`), highlighting topological defects, namely pentagons and heptagons in blue and red, respectively.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.tick_params(axis='both', labelsize=11)
ax.set_aspect('equal', 'box')

final_config = PV(dim, final_points, L, num_iter)
for idx, polytope in enumerate(final_config.voro.polytopes):
  facecolor = {5: "b", 6: "w", 7: "r"}
  ax.fill(*zip(*polytope[:, :2]), facecolor=facecolor[len(polytope)], edgecolor="k", linewidth=1.)
if num_points <= 2000:
  ax.scatter(final_points[:, 0], final_points[:, 1], s=2, color="k")
plt.show()

Let us compute the correlation length of the final point configuration, which represents the average hexagonal domain size.

In [ ]:
final_config.correlation_length()